# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [1]:
#vibo: отключение предупреждений
#import warnings
#warnings.filterwarnings('ignore')

#### vibo: подготовка данных

In [20]:
import numpy as np
from sklearn import datasets, model_selection, tree
from sklearn.metrics import mean_squared_error

#vibo: загружаем датасет
boston = datasets.load_boston()

#vibo: ключи датасета
boston.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [21]:
#vibo: датасет о стоимости жилья в Бостоне
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [22]:
X = boston.data
y = boston.target

#vibo: выделяем подвыборки для обучения и контроля качества (на тест последние 25% по улсловию задачи)
train_size = 0.75
train_size_number = int(y.shape[0] * train_size)
X_train = X[:train_size_number]
X_test = X[train_size_number:]
y_train = y[:train_size_number]
y_test = y[train_size_number:]

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

#### vibo: Решения Задания 2. Цель задания - реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь (величина шага постоянная 0.9)

In [23]:
#vibo: задаем число решающих деревьев (базовых алгоритмов)
N = 50

#vibo: список с базовыми алгоритмами
base_algorithms_list = []

#vibo: список с коэффициентами перед алгоритмами
coefficients_list = []

#vibo: шаг алгоритма
eta = 0.9

#vibo: список ошибок
mse_root_list = []

#vibo: функция которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке X
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

In [24]:
#vibo: задаем начальный алгоритм и обучаем его
regressor = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
#vibo: обучаем на train
regressor.fit(X_train, y_train)
#vibo: добавляем первым в список
base_algorithms_list.append(regressor)
#vibo: добавляем первый коэффициент в список
coefficients_list.append(eta)

#vibo: обучаем решающие деревья
for i in range(N-1):
    #vibo: рассчитываем сдвиг на train как истинный ответ минус предсказание
    #vibo: при работе функции gmb_predict берем из списка алгоритмов - начальный, считаем прогноз, домножаем на eta
    s = y_train - gbm_predict(X_train)
    #vibo: задаем алгоритм
    regressor = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    #vibo: обучаем алгоритм, приближаем к s
    regressor.fit(X_train, s)
    base_algorithms_list.append(regressor)
    coefficients_list.append(eta)
    #vibo: вычисляем ошибку на test
    mse_root = (mean_squared_error(y_test, gbm_predict(X_test)))**0.5 
    mse_root_list.append(mse_root)

In [5]:
mse_root_list

[4.671153714252575,
 4.902307279003964,
 4.964147618734462,
 5.20526450047539,
 5.135565602127196,
 5.240746955112687,
 5.289270262665005,
 5.474743913112658,
 5.487351059181247,
 5.493005009464344,
 5.486541185059441,
 5.485319403332965,
 5.491142418867942,
 5.4234289382521625,
 5.455189053902552,
 5.453903636946008,
 5.442535087357115,
 5.451577011033647,
 5.450957836386989,
 5.455454044759808,
 5.452830834182104,
 5.4534823020413565,
 5.456955189013637,
 5.4585969199592395,
 5.4597046298350955,
 5.458441954443328,
 5.459544166421452,
 5.474640780590228,
 5.455978129276078,
 5.456647037889836,
 5.456688163206112,
 5.455089494285212,
 5.454920050184027,
 5.454737355535651,
 5.4557529752777665,
 5.457222277871692,
 5.456585785858681,
 5.454982869324787,
 5.455803988401056,
 5.455699568702136,
 5.455320529878214,
 5.454721673397888,
 5.454973956294515,
 5.455111351969862,
 5.455731058970443,
 5.4557186100299795,
 5.455788482463016,
 5.455738872267899,
 5.455623403859612]

In [25]:
def write_answer(answer, filename):
    with open(filename, 'w') as fout:
        fout.write(str(answer))

write_answer(mse_root, "ans2.txt")

#### vibo: итого задание 2. RMSE = 5.4552 (ответ анализатора: Правильно. Далее вы увидите, что это на этом датасете это довольно неплохое качество.).

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

#### vibo: Решения Задания 3. Цель задания - реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь (величина шага уменьшается по формуле)

In [40]:
#vibo: задаем число решающих деревьев (базовых алгоритмов)
N = 50
#vibo: список с базовыми алгоритмами
base_algorithms_list = []
#vibo: список с коэффициентами перед алгоритмами
coefficients_list = []
#vibo: шаг алгоритма
eta = 0.9
#vibo: список ошибок
mse_root_list = []

In [41]:
#vibo: задаем начальный алгоритм и обучаем его
regressor = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
#vibo: обучаем на train
regressor.fit(X_train, y_train)
#vibo: добавляем первым в список
base_algorithms_list.append(regressor)
#vibo: добавляем первый коэффициент в список
coefficients_list.append(eta)

#vibo: обучаем решающие деревья
for i in range(N-1):
    #vibo: рассчитываем сдвиг на train как истинный ответ минус предсказание
    #vibo: при работе функции gmb_predict берем из списка алгоритмов - начальный, считаем прогноз, домножаем на eta
    s = y_train - gbm_predict(X_train)
    #vibo: задаем алгоритм
    regressor = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    #vibo: обучаем алгоритм, приближаем к s
    regressor.fit(X_train, s)
    base_algorithms_list.append(regressor)
    #vibo: уменьшаем коэффициент на каждом шаге по заданной формуле
    eta = 0.9 / (1 + (i+1))
    coefficients_list.append(eta)
    #vibo: вычисляем ошибку на test
    mse_root = (mean_squared_error(y_test, gbm_predict(X_test)))**0.5 
    mse_root_list.append(mse_root)

In [42]:
mse_root_list

[4.411594948336074,
 4.391183171661546,
 4.458775073285299,
 4.498661031729782,
 4.5168152101676124,
 4.550526944390747,
 4.609092468715132,
 4.6339875745933385,
 4.651558505714779,
 4.659211687810463,
 4.665846192244087,
 4.687756339977889,
 4.705212386904845,
 4.70888638920925,
 4.727160820532567,
 4.738601041184404,
 4.7434294144854805,
 4.747957771408691,
 4.754535803819599,
 4.760702807935853,
 4.76469042919606,
 4.764828870587975,
 4.763080541850576,
 4.763091173078234,
 4.762268473295419,
 4.758055256544697,
 4.768499657813316,
 4.7718046637409675,
 4.780389023483234,
 4.7765763172358175,
 4.783744654359199,
 4.7865280216634964,
 4.788442367935862,
 4.787212053443569,
 4.792422462196972,
 4.793468730432272,
 4.7994641171508885,
 4.804033058481719,
 4.8038723728476995,
 4.803086837269195,
 4.804028978406079,
 4.805470455205758,
 4.804790977053986,
 4.803709717063459,
 4.804773448498893,
 4.808927271733807,
 4.808240835910818,
 4.80887898153513,
 4.812550945781193]

In [43]:
def write_answer(answer, filename):
    with open(filename, 'w') as fout:
        fout.write(str(answer))

write_answer(mse_root, "ans3.txt")

#### vibo: итого задание 3. RMSE = 4.8125 (ответ анализатора: Верно. Обратите внимание, что более аккуратный выбор шага позволил понизить RMSE на тестовой выборке (если нет - попробуйте перезапустить алгоритм несколько раз - в среднем качество должно было улучшиться). Однако не стоит относиться к этому результату слишком доверчиво - небольшие изменения в формуле вычисления величины шага могут легко "сломать" этот эффект. Выбор хорошего шага в градиентном спуске всегда достаточно непростой вопрос - остается только порадоваться, что почти всегда можно использовать готовые реализации из библиотек.).

## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [51]:
#vibo: задаем число решающих деревьев (базовых алгоритмов)
N = 50
#vibo: список с базовыми алгоритмами
base_algorithms_list = []
#vibo: список с коэффициентами перед алгоритмами
coefficients_list = []
#vibo: шаг алгоритма
eta = 0.9
#vibo: список ошибок
mse_root_list = []

In [52]:
#vibo: задаем начальный алгоритм и обучаем его
regressor = tree.DecisionTreeRegressor(max_depth=10, random_state=42)
#vibo: обучаем на train
regressor.fit(X_train, y_train)
#vibo: добавляем первым в список
base_algorithms_list.append(regressor)
#vibo: добавляем первый коэффициент в список
coefficients_list.append(eta)

#vibo: обучаем решающие деревья
for i in range(N-1):
    #vibo: рассчитываем сдвиг на train как истинный ответ минус предсказание
    #vibo: при работе функции gmb_predict берем из списка алгоритмов - начальный, считаем прогноз, домножаем на eta
    s = y_train - gbm_predict(X_train)
    #vibo: задаем алгоритм
    regressor = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    #vibo: обучаем алгоритм, приближаем к s
    regressor.fit(X_train, s)
    base_algorithms_list.append(regressor)
    coefficients_list.append(eta)
    #vibo: вычисляем ошибку на test
    mse_root = (mean_squared_error(y_test, gbm_predict(X_test)))**0.5 
    mse_root_list.append(mse_root)

In [53]:
mse_root_list

[5.605855724198541,
 5.733275412567501,
 5.830622977259917,
 5.8149034264110435,
 5.804078213149917,
 5.819443313672626,
 5.862998614211611,
 5.804048123104557,
 5.813148418304858,
 5.791520374141209,
 5.804803706791228,
 5.817758725610122,
 5.798646880976972,
 5.796333731150376,
 5.799268991848334,
 5.798040968456205,
 5.800439420320781,
 5.8011659667460185,
 5.8009768800113655,
 5.805025626011457,
 5.805758282440396,
 5.807062424923722,
 5.80811400872747,
 5.8079300048299025,
 5.809262335951413,
 5.805821153036669,
 5.804069648260911,
 5.804894655970073,
 5.806323518336248,
 5.806820196236679,
 5.806607775272545,
 5.806516887427267,
 5.806227951011707,
 5.805924163417271,
 5.805781020112391,
 5.805723719764913,
 5.805823216256652,
 5.805394836340637,
 5.805430664696475,
 5.805382837028156,
 5.8054074868949455,
 5.805310057394385,
 5.805649724105423,
 5.805614713938634,
 5.805732516796667,
 5.805816955599942,
 5.80570927593649,
 5.805695568906455,
 5.805825928191022]

#### vibo: итого задание 4. Верно 2. из RMSE выше видно, что градиентный бустинг переобучается с ростом количества решающих деревьев. Верно 3. RMSE еще сильнее ростет с увеличением глубины решающих деревьев, композиция еще хуже, чем при увеличении числа решающих деревьев. (ответ анализатора: В самом деле, градиентный бустинг все больше подгоняется под данные с ростом числа деревьев, а рост глубины деревьев только ускоряет этот процесс. Начиная с некоторого момента алгоритм будет все больше переобучаться.)

In [54]:
write_answer('2 3', "ans4.txt")

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [58]:
from sklearn import linear_model

linear_regressor = linear_model.LinearRegression()
linear_regressor.fit(X_train, y_train)
mse_root = (mean_squared_error(y_test, linear_regressor.predict(X_test)))**0.5

In [59]:
mse_root

8.254979753549398

In [60]:
write_answer(mse_root, "ans5.txt")

#### vibo: итого задание 5. mse_root для линейной модели = 8.2550 сильно хуже, чем решающие деревья (ответ анализатора: Качество работы простого метода (линейной регрессии) оказалось хуже. Этот результат в некоторой степени завораживает: всего 1 деревьев, каждое из которых в каждом своем листе оценивает целевую зависимость некоторой константой, уже решили задачу регрессии лучше, чем линейная модель.)